# Second Homework: Network Optimization and Non-linear Models 

## Marta Almagro Fuello    
#### NIA : 100451979


## Fisrt Problem: Network optimization 

### a) Formulate the network optimization problem as a discrete model, identifying mathematically the variables and constraints associated with the network. Solve and interpret the solution.

The palace and fortress Alhambra in Granada is opening a new section that they have been restoring for the last 5 years. Before open it they want to plan every excursion and guide that is going to take place in those zones in order for the tourists to see all of the galleries available. They have calculated the price of each gallery, however they have also calculated the price of each path from the gallery to the rest, and after lots of studies they saw that they cannot have the same price for each of them because to go to some galleries they have to go through enormous beautiful gardens while other are just hallways that have no need of maintenance. They want to show the tourist all the galleries and expositions while getting the higher profit possible, which means going through the most beautiful paths.

<img src = "alhambra2.jpg" alt = "foto" />

They want to show the tourist all the galleries while getting the higher profit possible, in other words, maximize the price of the ticket to see all the galleries.

I have proposed this as a binary discrete model to optimize the whole problem and decide which path is in the final circuit so the company gains the maximum profit possible.
To reprensent the problem a network was created using the nodes as the galleries that have to be shown and the arcs as the paths that connect each gallery. There are prices associated to each path which in our case are going to be the costs, that will be added to the final price, when taking that path.

<img src = "map.jpg" alt = "mapa" />

In this model we have to keep in mind that they told us that each section can only be seeing one time because once the circuit is done they are going to close the paths that are not going to be used so the people cannot go to see again other sections. There is just one entry and just one exit, and any gallery cannot be repeated.


### Sets

 $G$ = nodes that represent the galleries of that section of the Alhambra
 
### Parameters

$Costs$ = cost for each path.

$Entry$ = where the excursion begings, the entry. $Entry \in [1]$

$Exit$ = where the excursion ends, the exit. $Exit \in [8]$


### Variables

$$x_{ij}=
\left\{\begin{array}{ll} 
1, & \text{The gallery $i$ is visited just before gallery $j$,}\\
0, & \text{else}\\
\end{array} \right.\quad$$

### Objective

Maximize the price of the ticket to visit Alhambra: 

$\max_{x_{ij}} \sum_{i,j \in G,i\neq j} c_{ij} x_{ij}$


### Constraints


They have to start in the entry, is not possible to start from other node:

$\sum_{j\in G} x_{1j} = 1$

The circuit has to end at gallery 8 (exit) because there is where the exit is placed:

$\sum_{i\in G} x_{i8} =1 $

There cannot be loops on each node (gallery):

$\sum_{i\in G} \sum_{j\in G,i=j} x_{ij} =0 $

The cycles have to be avoided:

$x_{ij} + x_{ji} \leq1$, $\forall i \in G $, $\forall j \in G $

There is only on path going out of each gallery and arriving to it:

Just one out =
$\sum_{j \in G} x_{ij} = 1$, $\forall i \in G $ except 8

Junt one in =
$\sum_{i\in G} x_{ij} = 1$, $\forall j \in G $ except 1

## Data

First we must import the data:

In [8]:
%%writefile alhambra.dat

param n := 8;
    
param cost :=
    1 1 0
    1 2 5
    1 3 7 
    1 4 1
    1 5 6
    1 6 2
    1 7 7
    1 8 5
    2 1 4
    2 2 0
    2 3 3
    2 4 4
    2 5 9 
    2 6 2
    2 7 9
    2 8 8
    3 1 6
    3 2 3
    3 3 0
    3 4 4
    3 5 6
    3 6 8
    3 7 9
    3 8 1    
    4 1 5
    4 2 2
    4 3 7
    4 4 0
    4 5 9
    4 6 3
    4 7 5
    4 8 8  
    5 1 3
    5 2 9
    5 3 2
    5 4 7
    5 5 0
    5 6 4
    5 7 9
    5 8 1
    6 1 6
    6 2 9
    6 3 2 
    6 4 4
    6 5 3
    6 6 0
    6 7 6
    6 8 9
    7 1 3
    7 2 7
    7 3 5
    7 4 5
    7 5 9
    7 6 3
    7 7 0
    7 8 5
    8 1 2
    8 2 5
    8 3 7
    8 4 3
    8 5 2
    8 6 6
    8 7 6
    8 8 0;

Writing alhambra.dat


## Model

Now we can create the model and start declaring the data:

In [9]:
%%writefile alhambra.py

# Import library
from pyomo.environ import *

# Create model
model = AbstractModel()

# Nodes and arc in the network
model.n = Param(within = NonNegativeIntegers)
model.i = Set(initialize = [1, 2, 3, 4, 5, 6, 7, 8]) 
model.j = Set(initialize = [1, 2, 3, 4, 5, 6, 7, 8]) 

# Initialize the source and the sink
model.entry = Set(initialize = [1]) # Entry
model.exit = Set(initialize = [8]) # Exit

# Initialize the costs of each arc
model.cost = Param(model.i, model.j) 

# Binary variable
model.x = Var(model.i, model.j, initialize = 0, domain=Binary)

# Maximize the flow 
def objective_rule(model):
    return sum(model.x[i,j] * model.cost[i,j] for i in model.i for j in model.j)

model.objective = Objective(rule = objective_rule, sense = maximize)
#model.objective.pprint()

# Constraint to avoid loops:
def avoid_loops(model):
    return sum(model.x[i,j] for i in model.i for j in model.j if i==j) == 0

model.AvoidLoops = Constraint(rule = avoid_loops)

# Constraint to avoid cycles:
def avoid_cycles(model, i, j):
    return model.x[i,j] + model.x[j,i] <= 1

model.AvoidCycles = Constraint(model.i, model.j, rule = avoid_cycles)

# Constraint to start in the entry
def start_entry(model):
    return sum(model.x[1,j] for j in model.j if j!=1) == 1

model.StartEntry = Constraint(rule = start_entry)

# Constraint to say that just one path go out of each gallery
# and in the exit none
def one_out(model, i):
    if i == 8:
        return sum(model.x[i,j] for j in model.j if j!=i) == 0
    else:
        return sum(model.x[i,j] for j in model.j if j!=i) <= 1
    
model.OneOut = Constraint(model.i, rule = one_out)
    
# Constraint to say that just one path go in of each gallery
# and in the entry none
def one_in(model, j):
    if j == 1:
        return sum(model.x[i,j] for i in model.i if j!=i) == 0
    else:
        return sum(model.x[i,j] for i in model.i if j!=i) <= 1

model.OneIn = Constraint(model.j, rule = one_in)

# Constraint to end in the exit
def end_exit(model):
    return sum(model.x[i,j] for i in model.i for j in model.exit) == 1

model.EndExit = Constraint(rule = end_exit)


Writing alhambra.py


In [10]:
#!cat alhambra.dat
#!cat alhambra.py

Finally the optimization problem is resolve with function .solve from the package 'glpk'

In [11]:
!pyomo solve --solver=glpk alhambra.py alhambra.dat

[    0.00] Setting up Pyomo environment
[    0.00] Applying Pyomo preprocessing actions
[    0.01] Creating model
[    0.04] Applying solver
[    0.11] Processing results
    Number of solutions: 1
    Solution Information
      Gap: 0.0
      Status: optimal
      Function Value: 57.0
    Solver results file: results.yml
[    0.11] Applying Pyomo postprocessing actions
[    0.11] Pyomo Finished


In [12]:
# Display
!cat results.yml

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 57.0
  Upper bound: 57.0
  Number of objectives: 1
  Number of constraints: 84
  Number of variables: 65
  Number of nonzeros: 256
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.023103952407836914
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------

## Conclusions and results

### Interpretation:

This problem has a feasible solution. The solution found for the *objective function* is **57** euros going through the following path:

$ Path$ | -------------------------------------------------------> $Cost$ 
 -----------------------|----
  Entry Gallery 3 | 7
  Gallery 3 Gallery 6 | 8 
  Gallery 6 Gallery 2 | 9
  Gallery 2 Gallery 7 | 9 
  Gallery 7 Gallery 5 | 9 
  Gallery 5 Gallery 4 | 7 
  Gallery 4 Gallery 8 | 8 
 Total price of the ticket | 57€
 
 Checking the data we can observe that the solution makes sense because the costs in the table below are between 7 and 9 which are actually the highest values of the costs.
In this map I traced the paths choosen and we confirm that it starts in 1 and ends in 8 going through every gallery just once.

<img src = "solution_map.jpg" alt = "Companies" />

# Second problem: Non-linear optimization

### b) Formulate (mathematically) and solve a non-linear optimization problem based on real (or realistic) world data. Interpret the solution.

In a physics laboratory at the university of Massachusets they are doing an experiment related with the energy of an object that has a rest mass ($m_{0}$) and a speed ($v$), among other factors. In this stage of the experiment they want to see how can they get the maximum energy of an object with a total mass that cannot be greater than half a ton.

The formula that determines the energy is the following:

$ E = \frac{1}{2} m_{0} v^2 + m_{0} c^2 $

Where c is the speed of light which a constant equal to $3 * 10^8$m/s


This problem would be really easy just like that, however in the laboratory they have some limitations:
 - The lab is not very big and the object has a range of 250 meters so the speed cannot be more than:
    
  $$ 250 \geq v * 30 $$   such that $$ d = v*t $$ 
     
   taking into account that they want the experiment to last 30 seconds and it has a uniform speed (not aceleration)
 - Another limitation is a physic limitation which is the formula that states that the total mass is:  
     
 $$ M = m_{0}^2 + M*0.1 $$
   
 So if the total mass is at most 500 kg:
  $$ 500 \geq m_{0}^2 + 50 $$


### Parameters

Distance = at most 250 meters.

Time = 30 seconds

Speed of light = $3 * 10^8$ m/s


### Variables

 $m_{0}$ = rest mass of the object in kg.           
 $v$ = speed of the object in m/s.

### Objective

Maximize the total energy of the object: 

$ E(m_{0},v) = \frac{1}{2} m_{0} v^2 + m_{0} c $

$\max_{m_{0},v} E(m_{0},v)$


### Constraints


- The mass is given by the inequation:

  $$ 500 \geq m_{0}^2 + 50 $$

- The speed is limited by the time and distance:

$$ 250 \geq v * 30 $$

- The speed and rest mass have to be more than 0


$$ v > 0  $$  $$ m_{0} > 0  $$


## Model

### Solution with Pyomo

In [2]:
#conda install -c conda-forge pyomo

In [1]:
#conda install -c conda-forge ipopt

In [3]:
# Import library
from pyomo.environ import *
from pyomo.opt import SolverFactory

import numpy as np

# Create model
model = ConcreteModel()

# Define the variables m and v
model.m = Var(within = NonNegativeIntegers, initialize = 1.0)
model.v = Var(within = NonNegativeIntegers, initialize = 1.0)

In [4]:
# Objective function
model.obj = Objective(expr = (1/2) * model.m * (model.v**2) + (model.m *  3.0 * 10**16),
                            sense = maximize)

# Define the constraints
model.cons2 = Constraint(expr = 250 - model.v * 30 >= 0 )
model.cons3 = Constraint(expr = -model.m**2 + 450  >= 0)

In [5]:
# Obtain the solution
solver = SolverFactory("ipopt") # define the solver
results = solver.solve(model) # solve

# Display solution
display(model.obj)

print('m0 = ', model.m())
print('v = ', model.v())

obj : Size=1, Index=None, Active=True
    Key  : Active : Value
    None :   True : 6.36396106249874e+17
m0 =  21.21320354166244
v =  8.333333345707237


### Solution with Scipy 

In [6]:
from scipy.optimize import minimize


# Define the objective function
def func(x): # x = (m, v)
    return -((1/2) * x[0] * x[1]**2 + x[0] * 10**16)

# Define the jacobian of the objective function
def func_deriv(x):
    return np.array([ -((1/2)* x[1]**2 + 10**16), -(x[0] * 10**16) ])

# Define the constraints and its jacobian arrays
cons = (
    {'type': 'eq', 'fun' : lambda x: np.array([250 - 30 * x[1]]), 'jac' : lambda x: np.array([0.0, -30.0])},
    {'type': 'eq','fun' : lambda x: np.array([-x[0]**2 + 450 ]),'jac' : lambda x: np.array([-2*x[0], 0.0])},
    {'type': 'ineq','fun' : lambda x: np.array([x[0]-1]),'jac' : lambda x: np.array([1.0, 0.0])},
    {'type': 'ineq','fun' : lambda x: np.array([x[1]-1]),'jac' : lambda x: np.array([0.0, 1.0])} )  

#  # Solve
result = minimize(func, x0 = [1.0, 1.0], jac = func_deriv, 
               constraints = cons, method = 'SLSQP',  options={'disp': True})

# Display the results
print(result.x)


Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.12132034355965e+17
            Iterations: 5
            Function evaluations: 7
            Gradient evaluations: 5
[21.21320344  8.33333333]


#### Lagrange Multipliers.

In [7]:
# Here we are going to mininmixe
result1 = minimize(func, x0 = [1.0,1.0], jac = func_deriv, 
              constraints = cons, method = 'trust-constr')

# Print the values for v and m
print("x = ",result1.x,"\n")

# Print the lagrange multipliers
print("Lagrange multipliers = ", result1.v)

x =  [21.21320344  8.33333333] 

Lagrange multipliers =  [array([-7.07106781e+15]), array([-2.3570226e+14]), array([-2.53299781e-09]), array([-6.98181818e-09])]


/opt/conda/lib/python3.10/site-packages/scipy/optimize/_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


## Conclusion



We conclude that the maximum point of this problem is with rest mass equal to 21.2132 kg and speed equal to  8.3333 m/s. The total energy generated by that object is going to be 6.36396106249874e+17.
And that the Lagrange Multipliers are -7.07106781e+15, -2.3570226e+14, -2.53299781e-09 and-6.98181818e-09. 

There can be seen that doing the same problem with Scipy and Pyomo we obtain the same results which confirms that the problem formulation for each method is the right one.

To check that the constraints are taken into account:

In [8]:
500 >= 21.2132**2 + 50
250 >= 8.3333 * 30

True